<a href="https://colab.research.google.com/github/Ranjan00/Avinash-Ranjan/blob/master/mining_yeardetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-akr')


In [5]:
# Install required packages
!pip install geemap --quiet
import geemap
import ee
ee.Initialize(project='ee-akr')


In [10]:
!pip install pycrs


  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32686 sha256=f1c799aea26af3950bef8ecd4254905dc46cf017b3d8d85c107b3caa7a8f8ed1
  Stored in directory: /root/.cache/pip/wheels/5f/ad/a3/183ed754d7698fc15a2eb153705e05d05a0d97f3331293ce48
Successfully built pycrs


In [14]:
from google.colab import files
uploaded = files.upload()


Saving mining_points.zip to mining_points.zip


In [15]:
import zipfile
import os

with zipfile.ZipFile("mining_points.zip", 'r') as zip_ref:
    zip_ref.extractall("mining_points")

shapefile_path = "mining_points/mining_points.shp"


In [16]:
import geemap

mining_fc = geemap.shp_to_ee(shapefile_path)


In [17]:
years = list(range(1984, 2025))  # NDVI from 1984 to 2024


In [18]:
# Helper to select NDVI from Landsat SR (Landsat 5, 7, 8, 9)
def get_landsat_ndvi(year):
    start = f'{year}-01-01'
    end = f'{year}-12-31'

    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate(start, end).filterBounds(mining_fc)
    l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterDate(start, end).filterBounds(mining_fc)
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start, end).filterBounds(mining_fc)
    l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(start, end).filterBounds(mining_fc)

    def sr_to_ndvi(img):
        ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']) \
            .rename('NDVI') \
            .copyProperties(img, img.propertyNames())
        return ndvi

    def apply_sr_bandnames(img):
        b_map = {
            'SR_B3': 'Red', 'SR_B4': 'NIR', # L5/L7
            'SR_B4': 'Red', 'SR_B5': 'NIR', # L8/L9
        }
        return img

    ndvi_l5 = l5.map(sr_to_ndvi)
    ndvi_l7 = l7.map(sr_to_ndvi)
    ndvi_l8 = l8.map(sr_to_ndvi)
    ndvi_l9 = l9.map(sr_to_ndvi)

    merged = ndvi_l5.merge(ndvi_l7).merge(ndvi_l8).merge(ndvi_l9)
    return merged.median().set('year', year)


In [22]:
import os

# List files inside the folder
os.listdir("mining_points")


['mining_points.dbf',
 'mining_points.shx',
 'mining_points.prj',
 'mining_points.shp']

In [25]:
# Unzip and list
!unzip mining_points.zip -d mining_points
import os
print(os.listdir("mining_points"))  # Should show .shp, .shx, etc.

# Import shapefile into Earth Engine
import geemap
shapefile_path = "mining_points/mining_points.shp"
mining_fc = geemap.shp_to_ee(shapefile_path)


Archive:  mining_points.zip
replace mining_points/mining_points.shx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mining_points/mining_points.shx  
replace mining_points/mining_points.dbf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mining_points/mining_points.dbf  
replace mining_points/mining_points.prj? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mining_points/mining_points.prj  
replace mining_points/mining_points.shp? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
['mining_points.dbf', 'mining_points.shx', 'mining_points.prj', 'mining_points.shp']


In [26]:
import geemap

shapefile_path = "mining_points/mining_points.shp"
mining_fc = geemap.shp_to_ee(shapefile_path)


In [27]:
# Upload KMZ (points) as EE FeatureCollection
mining_fc = geemap.shp_to_ee("mining_points.shp")  # Make sure points shapefile is used

# Sample NDVI at each point for each year
samples_per_year = []
for i, y in enumerate(years):
    ndvi_image = get_landsat_ndvi(y)
    sampled = ndvi_image.sampleRegions(
        collection=mining_fc,
        scale=30,
        geometries=True
    ).map(lambda f: f.set('year', y))
    samples_per_year.append(sampled)

# Merge yearly samples into one FeatureCollection
all_samples = ee.FeatureCollection(samples_per_year).flatten()


The input shapefile could not be found.


In [31]:
# Install needed packages (run once)
!pip install geemap geopandas

import ee
import geemap
import geopandas as gpd
import time
import os

# Authenticate and initialize EE
ee.Authenticate()
ee.Initialize(project='ee-akr')

# Set years range
years = list(range(1984, 2025))

# Unzip uploaded mining points shapefile (make sure you uploaded mining_points.zip)
import zipfile
with zipfile.ZipFile('mining_points.zip', 'r') as zip_ref:
    zip_ref.extractall('./mining_points')

shp_path = './mining_points/mining_points.shp'

# Load mining points as GeoDataFrame
gdf = gpd.read_file(shp_path)

# Convert GeoDataFrame to EE FeatureCollection
mining_fc = geemap.shp_to_ee(shp_path)

print(f"Loaded {len(gdf)} mining points.")

# Function to get annual median Landsat NDVI for a given year
def get_landsat_ndvi(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    # Use Landsat 5,7,8 collection with Surface Reflectance (depending on year)
    # Combine collections for seamless transition:
    collection = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
                  .filterDate(start_date, end_date)
                  .merge(ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
                         .filterDate(start_date, end_date))
                  .merge(ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                         .filterDate(start_date, end_date)))

    # Filter out images with cloud cover over 50%
    collection = collection.filter(ee.Filter.lt('CLOUD_COVER', 50))

    # Calculate NDVI = (NIR - Red) / (NIR + Red)
    def add_ndvi(img):
        ndvi = img.normalizedDifference(['B4', 'B3']).rename('NDVI')  # Landsat SR bands B4 = NIR, B3 = Red
        return img.addBands(ndvi)

    collection = collection.map(add_ndvi)

    # Median NDVI image for the year
    ndvi_median = collection.select('NDVI').median().set('year', year)

    return ndvi_median

# Sample NDVI for each mining point and each year sequentially (to avoid concurrency errors)
samples_per_year = []

for y in years:
    print(f"Sampling NDVI for year: {y}")
    ndvi_image = get_landsat_ndvi(y)

    # Sample NDVI at mining points, include geometry for tracking
    sampled = ndvi_image.sampleRegions(
        collection=mining_fc,
        scale=30,
        geometries=True
    ).map(lambda f: f.set('year', y))

    samples_per_year.append(sampled)

    # Wait 10 seconds before next year to avoid concurrency limit
    time.sleep(10)

# Merge all samples into one FeatureCollection
all_samples = ee.FeatureCollection(samples_per_year).flatten()

# Get all sampled features as GeoJSON (this can be slow for very large data)
print("Downloading sampled NDVI data...")
features_dict = all_samples.getInfo()

# Extract NDVI timeseries per mining point (by unique ID)
# Assuming mining points have an 'id' or 'system:index' property

# Map to store NDVI per point
from collections import defaultdict
point_ndvi = defaultdict(lambda: [None] * len(years))

# Build a mapping from years to index
year_index = {y: i for i, y in enumerate(years)}

for feat in features_dict['features']:
    props = feat['properties']
    geom = feat['geometry']
    ndvi_val = props.get('NDVI', None)
    year = props.get('year', None)
    # Use 'system:index' as unique point id (or replace with your own ID field)
    point_id = feat['id']

    if ndvi_val is not None and year in year_index:
        idx = year_index[year]
        point_ndvi[point_id][idx] = ndvi_val

# Now apply your 3-year consecutive NDVI decline detection per point
def detect_decline(ndvi_list, years):
    decline_start_year = None
    for i in range(len(ndvi_list) - 3):
        # Check if strictly decreasing for 3 consecutive years
        if ndvi_list[i] is not None and ndvi_list[i+1] is not None and ndvi_list[i+2] is not None and ndvi_list[i+3] is not None:
            if ndvi_list[i] > ndvi_list[i+1] > ndvi_list[i+2] > ndvi_list[i+3]:
                decline_start_year = years[i]
                break
    return decline_start_year

# Add 'mining_start_year' to GeoDataFrame
mining_start_years = []

# We need a mapping from system:index to GeoDataFrame row index
# system:index from EE features might not correspond to row index - check your shapefile's ID field for unique ID
# For simplicity, assume the order is the same and IDs are '0', '1', ... (check this in your actual data)

for i, row in gdf.iterrows():
    point_id = str(i)  # Change this if you have a unique ID field in your shapefile
    ndvi_list = point_ndvi.get(point_id, [None] * len(years))
    decline_year = detect_decline(ndvi_list, years)
    mining_start_years.append(decline_year if decline_year is not None else -1)

gdf['mining_start_year'] = mining_start_years

print(gdf[['mining_start_year']])

# Export updated shapefile as ZIP
output_folder = './updated_mining_points'
os.makedirs(output_folder, exist_ok=True)

output_path = os.path.join(output_folder, 'mining_points_updated.shp')
gdf.to_file(output_path)

# Zip the output shapefile for download
import shutil
shutil.make_archive('mining_points_updated', 'zip', output_folder)

print("Updated shapefile saved and zipped as mining_points_updated.zip")


Loaded 3161 mining points.
Sampling NDVI for year: 1984


/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for LANDSAT/LT05/C01/T1_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/reference/removed_datasets#landsat_collection_1

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for LANDSAT/LE07/C01/T1_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/reference/removed_datasets#landsat_collection_1

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for LANDSAT/LC08/C01/T1_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.


Sampling NDVI for year: 1985
Sampling NDVI for year: 1986
Sampling NDVI for year: 1987
Sampling NDVI for year: 1988
Sampling NDVI for year: 1989
Sampling NDVI for year: 1990
Sampling NDVI for year: 1991
Sampling NDVI for year: 1992
Sampling NDVI for year: 1993
Sampling NDVI for year: 1994
Sampling NDVI for year: 1995
Sampling NDVI for year: 1996
Sampling NDVI for year: 1997
Sampling NDVI for year: 1998
Sampling NDVI for year: 1999
Sampling NDVI for year: 2000
Sampling NDVI for year: 2001
Sampling NDVI for year: 2002
Sampling NDVI for year: 2003
Sampling NDVI for year: 2004
Sampling NDVI for year: 2005
Sampling NDVI for year: 2006
Sampling NDVI for year: 2007
Sampling NDVI for year: 2008
Sampling NDVI for year: 2009
Sampling NDVI for year: 2010
Sampling NDVI for year: 2011
Sampling NDVI for year: 2012
Sampling NDVI for year: 2013
Sampling NDVI for year: 2014
Sampling NDVI for year: 2015
Sampling NDVI for year: 2016
Sampling NDVI for year: 2017
Sampling NDVI for year: 2018
Sampling NDVI 

EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LC08/C01/T1_SR' not found (does not exist or caller does not have access).